In [3]:
from torchvision.datasets import MNIST
from torchvision import transforms
import torch
import os

train_data = MNIST(
    root=os.path.join("dataset", "mnist"),
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)
test_data = MNIST(
    root=os.path.join("dataset", "mnist"),
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import torch
from torch import nn
from collections import OrderedDict

class Model(nn.Module):
    def __init__(self, in_features, out_features=10) -> None:
        super().__init__()
        self.in_features = in_features
        self.layer1 = nn.Linear(in_features=self.in_features, out_features=256)
        self.layer2 = nn.Linear(in_features=256, out_features=128)
        self.layer3 = nn.Linear(in_features=128, out_features=64)
        self.layer4 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x: torch.Tensor):
        x = x.view(-1, self.in_features)
        x = self.layer1(x)
        x = nn.functional.relu(x)
        x = self.layer2(x)
        x = nn.functional.relu(x)
        x = self.layer3(x)
        x = nn.functional.relu(x)
        x = self.layer4(x)
        return x


model = Model(28 * 28).to(device=device)
model


Model(
  (layer1): Linear(in_features=784, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=64, bias=True)
  (layer4): Linear(in_features=64, out_features=10, bias=True)
)

In [5]:
from torch import optim
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()


In [6]:
from torch.utils.data import DataLoader

data_loader = DataLoader(
    dataset=train_data, batch_size=64, shuffle=True, drop_last=True
)


In [7]:
epochs = 10
for epoch in range(1, epochs + 1):
    for X, y in data_loader:
        X = X.to(device=device)
        y = y.to(device=device)

        train_preds = model(X)
        loss = criterion(train_preds, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"epoch {epoch:3}: {loss.item()}")


epoch   1: 0.278544545173645
epoch   2: 0.10389763116836548
epoch   3: 0.007537279278039932
epoch   4: 0.03480694815516472
epoch   5: 0.09433475881814957
epoch   6: 0.005727361887693405
epoch   7: 0.0313013531267643
epoch   8: 0.07200540602207184
epoch   9: 0.11212671548128128
epoch  10: 0.07564185559749603


In [10]:
test_preds = model(test_data.data.to(device=device, dtype=torch.float))
test_labels = test_data.targets
from sklearn.metrics import classification_report

print(
    classification_report(
        test_preds.to(device="cpu").detach().numpy().argmax(axis=1), test_labels,
        digits=4
    )
)


              precision    recall  f1-score   support

           0     0.9939    0.9828    0.9883       991
           1     0.9938    0.9921    0.9930      1137
           2     0.9893    0.9632    0.9761      1060
           3     0.9802    0.9659    0.9730      1025
           4     0.9695    0.9835    0.9764       968
           5     0.9843    0.9659    0.9750       909
           6     0.9781    0.9884    0.9832       948
           7     0.9874    0.9713    0.9793      1045
           8     0.9661    0.9843    0.9751       956
           9     0.9445    0.9917    0.9675       961

    accuracy                         0.9789     10000
   macro avg     0.9787    0.9789    0.9787     10000
weighted avg     0.9792    0.9789    0.9789     10000



https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html